In [1]:
''' 

Submitter : JESOON KANG, 20170937
Date : 2019. 10. 


    Assignment 4. 

-   -


'''
import matplotlib.pyplot as plt
import math
import numpy as np
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

import torchvision
import os




In [2]:
# predict Function. if h(x) returns >=0.5, set to 1. other cases, set to 0.
def predict(h, labels) :
    mount = len(h)
    correct = 0
    
    for i in range(0,mount) :
        if h[i] >= 0.5 :
            if labels[i] == 1 :
                correct +=1
        else :
            if labels[i] == 0 :
                correct +=1
    return correct * (1/mount)

#### Section 2 END ####

In [270]:

#### Section 1. #### This Section is bringed Data_import_ex.py file.

# Image Data import & resize

transform = transforms.Compose([#transforms.Resize((256,256)),  
                                transforms.Grayscale(),		# the code transforms.Graysclae() is for changing the size [3,100,100] to [1, 100, 100] (notice : [channel, height, width] )
                                transforms.ToTensor(),])

#train_data_path = 'relative path of training data set'
train_data_path = '../data/horse-or-human/train'
trainset = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
# if shuffle=True, the data reshuffled at every epoch 
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=True, num_workers=0)  


validation_data_path = '../data/horse-or-human/validation'
valset = torchvision.datasets.ImageFolder(root=validation_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
valloader = torch.utils.data.DataLoader(valset, batch_size=1, shuffle=False, num_workers=0)  

#### Section 1 END ####


In [276]:
#### Section 3 START ####
# Section 3 includes Data Pre-processing. ReDesign datasets to easy calculate

# Data reconstruct. vectorize

# Each Image file will be stored shape of a row
training_vectorized = []
training_labels = []


tmp = 0
# Training data vectorizing
for i, data in enumerate(trainloader) :
    train_data = []
    inputs, labels = data
#    print(tmp)
    tmp += 1
    for u in inputs :
        for col in u[0] :
            train_data += list(col)
    training_vectorized.append(train_data)
    training_labels.append([labels])

training_vectorized = torch.Tensor(training_vectorized)
training_labels = torch.Tensor(training_labels)
print("train set is loaded")



train set is loaded


In [277]:
print(training_vectorized)

tensor([[0.1216, 0.1137, 0.2235,  ..., 0.6157, 0.6196, 0.5882],
        [0.7647, 0.7608, 0.7608,  ..., 0.7569, 0.7647, 0.7686],
        [0.2549, 0.3333, 0.3725,  ..., 0.5373, 0.5020, 0.5098],
        ...,
        [0.7412, 0.7373, 0.7373,  ..., 0.7137, 0.7137, 0.6941],
        [0.7569, 0.7608, 0.7647,  ..., 0.3529, 0.4510, 0.4784],
        [0.7412, 0.7647, 0.8157,  ..., 0.1529, 0.1882, 0.1882]])


In [280]:

validation_vectorized = []
validation_labels = []

# Validation data vectorizing
for i, data in enumerate(valloader) :
    val_data = []
    inputs, labels = data
    for u in inputs :
        for col in u[0] :
            val_data += list(col)
    validation_vectorized.append(val_data)
    validation_labels.append([labels])
validation_vectorized = torch.Tensor(validation_vectorized)
validation_labels = torch.Tensor(validation_labels)

#### Section 3 END ####



In [281]:
print(validation_vectorized)

tensor([[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        ...,
        [1.0000, 1.0000, 1.0000,  ..., 0.9961, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 0.9961, 0.9961, 0.9961],
        [1.0000, 1.0000, 1.0000,  ..., 0.9961, 0.9961, 0.9961]])


In [282]:

# sigmoid Function.
def sigmoid(z) :
    return 1 / (1 + torch.pow(math.e,-z/100))

def get_activation(z,type) :
    if (type == 0) :
        return sigmoid(z)
    else :
        print("Error, get_activation")
        return 0

def get_derv_activation(z,type) :
    if (type == 0) :
        return sigmoid(z)*(1-sigmoid(z))
    else :
        print("Error, get_derv_activation")
        return 0                                                                  

In [283]:

# log variables setting : to record statements
log_training_loss = []                                                                                                                                                                                  
log_validation_loss = []                                                                                                                                                                                                                                                                                                                                                                                       
log_training_acc = []
log_validation_acc = []


In [287]:
def get_loss(y,a) :
    #print(y,a)
    ret = -(torch.div(y,a) - torch.div(1-y,1-a))
    
    return ret


class ML :
    def __init__(self,t_data,t_yh,v_data,v_yh,L1_size,L2_size,L3_size,activation_type,lr) :
        self.t_data = t_data
        self.t_data_size = self.t_data.shape[0]
        self.feature_size = self.t_data.shape[1]
        self.t_yh = t_yh
        self.v_data = v_data
        self.v_yh = v_yh
        self.l1_size = L1_size
        self.l2_size = L2_size
        self.l3_size = L3_size
        self.batch_size = 256
        self.t_batch_idx = 0
        self.epoch = 0
        self.lr = lr
        self.act_type = activation_type
        self.init_weights()
        
        print("ML Object initialized")
        self.iter = 0
        

    def get_training_batch(self) :
        self.idx_1 = self.t_batch_idx * self.batch_size
        self.idx_2 = self.idx_1 + self.batch_size
        self.idx_3 = 0
        if self.idx_2 > self.t_data_size :
            #print(self.idx_1,self.idx_2,self.idx_3)
            self.idx_3 = self.idx_2 - self.t_data_size
            self.idx_2 = self.t_data_size
            
            tmp_tensor = self.t_data[self.idx_1:self.idx_2].clone().detach()
            tmp_tensor_2 = self.t_data[0:self.idx_3].clone().detach()
            
            
            ret = torch.cat((tmp_tensor,tmp_tensor_2),0)
            
            if (ret.shape[0] != self.batch_size) :
                print("ERROR at get batch, %s"%(ret.shape[1]))
            self.t_batch_idx = 0
            self.epoch += 1
        else :
            self.t_batch_idx += 1
            ret = self.t_data[self.idx_1:self.idx_2].clone().detach()

        
        return ret
    def get_t_yh_batch(self) :
        if self.idx_3 == 0 :
            ret = self.t_yh[self.idx_1:self.idx_2].clone().detach()
        else :
            tmp_tensor = self.t_yh[self.idx_1:self.idx_2].clone().detach()
            tmp_tensor_2 = self.t_yh[0:self.idx_3].clone().detach()
            
            ret = torch.cat((tmp_tensor,tmp_tensor_2),0)
        return ret

    
    def init_weights(self) :
        self.w_1 = torch.FloatTensor(self.l1_size,self.feature_size).uniform_(-1,1)
        self.b_1 = torch.FloatTensor(self.batch_size,1).uniform_(-1,1)
        
        #torch.FloatTensor(a, b).uniform_(r1, r2)
        
        self.w_2 = torch.FloatTensor(self.l2_size,self.l1_size).uniform_(-1,1)
        self.b_2 = torch.FloatTensor(self.batch_size,1).uniform_(-1,1)

        self.w_3 = torch.FloatTensor(self.l3_size,self.l2_size).uniform_(-1,1)
        self.b_3 = torch.FloatTensor(self.batch_size,1).uniform_(-1,1)

        

        
    def training(self) :
    
        while(True) :
            self.t_data_batch = self.get_training_batch()
            self.z_1 = torch.matmul(self.t_data_batch,self.w_1.T) + self.b_1
            
            self.a_1 = get_activation(self.z_1,self.act_type)
            
            
            self.z_2 = torch.matmul(self.a_1,self.w_2.T) + self.b_2
            self.a_2 = get_activation(self.z_2,self.act_type) # 1027 x 50

            self.z_3 = torch.matmul(self.a_2,self.w_3.T) + self.b_3
            self.a_3 = get_activation(self.z_3,self.act_type) # 1027 x 1

            self.t_yh_batch = self.get_t_yh_batch()

            acc = self.get_acc(self.a_3,self.t_yh_batch)
            

            print("iter : %s, epoch : %s, loss : %s, acc : %s"%(self.iter,self.epoch,0,acc))

            self.update_weights(self.t_yh_batch,self.a_3)
            print(np.array(get_loss(self.t_yh_batch, self.a_3)).mean())
            self.validation()
            #break
            if self.epoch > 1000000 :
                break
    
    def validation(self) :
        self.v_data_batch = self.v_data
        self.v_yh_batch = self.v_yh
        
        self.z_1 = torch.matmul(self.v_data_batch,self.w_1.T) + self.b_1
        self.a_1 = get_activation(self.z_1,self.act_type) # Activation 함수 적용
        self.z_2 = torch.matmul(self.a_1,self.w_2.T) + self.b_2
        self.a_2 = get_activation(self.z_2,self.act_type) # Activation 함수 적용
        self.z_3 = torch.matmul(self.a_2,self.w_3.T) + self.b_3
        self.a_3 = get_activation(self.z_3,self.act_type) # y^과 같음
        
        
        
        acc = self.get_acc(self.a_3,self.v_yh_batch)
        print("val acc : %s"%(acc))
    
        
    def update_weights(self,t_y,a_3) :
        #print("t_y:",t_y,"z_3",z_3)
        error_wb3 = -(torch.div(t_y,a_3) - torch.div(1-t_y,1-a_3))
        d_a_3 = error_wb3*get_derv_activation(self.a_3,self.act_type) #         
        d_w_3 = torch.matmul(d_a_3,self.a_2.T)
        d_b_3 = torch.sum(d_a_3, dim=1, keepdim=True) / self.a_3.shape[1]
        
        #########
        
        error_wb2 = torch.matmul(d_a_3,self.w_3)
        
        d_a_2 = error_wb2*get_derv_activation(self.a_2,self.act_type)
        
        d_w_2 = torch.matmul(d_a_2.T,self.a_1)
        d_b_2 = torch.sum(d_a_2, dim=1,keepdims=True) / self.a_2.shape[1]
        
   
        error_wb1 = torch.matmul(d_a_2,self.w_2)
        d_a_1 = error_wb1*get_derv_activation(self.a_1,self.act_type)
        d_w_1 = torch.matmul(d_a_1.T,self.t_data_batch)
        d_b_1 = torch.sum(d_a_1, dim=1,keepdims=True) / self.a_1.shape[1]
        
        self.w_3 += -self.lr*d_w_3
        self.b_3 += -self.lr*d_b_3
        
        self.w_2 += -self.lr*d_w_2
        self.b_2 += -self.lr*d_b_2
        self.w_1 += -self.lr*d_w_1
        self.b_1 += -self.lr*d_b_1
        #print(b_3,b_2,b_1)
        
    def get_acc(self,yhat,y) :
        count = 0
        
        for a,b in zip(yhat,y) :
            if a >= 0.5 :
                if b == 1 :
                    count+=1
            else :
                if b == 0:
                    count +=1
        
        return count / len(yhat)

 

In [288]:
t_data = torch.FloatTensor(training_vectorized)
t_yh = torch.FloatTensor(training_labels)
v_data = torch.FloatTensor(validation_vectorized)
v_yh = torch.FloatTensor(validation_labels)
print(t_data.shape,t_yh.shape,v_data.shape,v_yh.shape)

activation_type = 0 # 0 = sigmoid
learningRate = 0.1
machine = ML(t_data,t_yh,v_data,v_yh, 100,50,1,0,learningRate)

torch.Size([1027, 10000]) torch.Size([1027, 1]) torch.Size([256, 10000]) torch.Size([256, 1])
ML Object initialized


In [289]:
machine.training()

iter : 0, epoch : 0, loss : 0, acc : 0.53125


RuntimeError: size mismatch, m1: [256 x 1], m2: [50 x 256] at ../aten/src/TH/generic/THTensorMath.cpp:752